In [31]:
import requests
import os
import pandas as pd
import json
from time import sleep
from dotenv import load_dotenv

Liste des réseaux que je veux étudier

In [32]:
LISTE_RESEAUX_ETUDIES =["network:SNCF:DBS",
                        "network:SNCF:EUROSTAR",
                        "network:SNCF:IC",
                        "network:SNCF:ICN",
                        "network:SNCF:FR:Branding::a80c322c-12b1-4273-9413-e2df5dd6dce8:",
                        "network:SNCF:TGVOUIGO",
                        "network:SNCF:OUIGO_TC",
                        "network:SNCF:SNF",
                        "network:SNCF:TER",
                        "network:SNCF:FR:Branding::6dfb1ec4-50b5-42a8-a531-a792b1ea6f2e:",
                        "network:SNCF:FR:Branding::5c5f14b9-bff2-4f94-8853-0f621bf1a1cf",
                        "network:SNCF:OUI",
                        "network:SNCF:LYR",
                        "network:SNCF:TN",
                        "network:additional_service"
                        ]

In [33]:
# Recupération de la clé api depuis env
load_dotenv()
api_key = os.getenv("API_KEY")

In [34]:
def requete_reseau(code,api_key):
    '''Requete une liste de gare à une page spécifique'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:{code}/networks", auth=(api_key,""))
    sleep(1)
    return reponse.json()

In [35]:
def requete_info_gare(page):
    url = f"https://ressources.data.sncf.com/api/explore/v2.1/catalog/datasets/frequentation-gares/records?limit=50&offset={page*50}"
    reponse = requests.get(url).json()
    return reponse

In [36]:
def enregistrer_json(data,nom_fichier):
    '''Cette fonction enregistre un fichier json'''
    with open(nom_fichier,"w") as fc:
        json.dump(data,fc)

In [37]:
def calcul_nombre_page():
        
    premiere_requete = requete_info_gare(0)
    nb_gare = premiere_requete["total_count"]
    nb_page = nb_gare // 100
    if nb_gare % 100 != 0:
        nb_page += 1

    return nb_page

In [38]:
def recuperer_toutes_pages(nb_pages):
    '''Cette fonction fait une requete api pour chaque page et stocke les resultats dans une liste'''
    #Initie liste gare
    liste_info_gare = []
    # Requete api pour chaque page
    for page in range(nb_pages):
        #recupère les données de la clé results
        data_page = requete_info_gare(page)["results"]
        #ajoute les données à la liste, element par element
        for item in data_page:
            liste_info_gare.append(item)
    return liste_info_gare  

In [39]:
def extraction_network(data):
    try:
        # Extraction de tous les reseau d'une gare
        tous_reseaux = data["networks"]
        return tous_reseaux

    except KeyError:
        return None


In [40]:
def ajout_reseau(data,liste_gares,cle_api):
    '''Creer une liste de gare en ajoutant un dictionnaire de réseau à chaque gare'''

    compteur = 0
    nb_gare = len(data)

    for item in data:
        # Extraction du code uic
        code_gare = item["code_uic"]
        # Requete pour récupérer les infos réseau
        resultat = requete_reseau(code_gare,cle_api)
        # Extraction des réseaux
        liste_reseaux = extraction_network(resultat)
        # Ajout d'un ensemble clé valeur avec la liste des réseaux
        item["networks"]=liste_reseaux
        # Ecriture de toutes les informations de la gare dans une liste
        liste_gares.append(item)
        #Incrémentation du compteur
        compteur += 1
        # Si le compteur est multiple de 10 affiche un message
        if compteur % 10 == 0:
                print(f"gare n°{compteur} sur {nb_gare}")

    
    return liste_gares


In [41]:
def trie_gare(data,LISTE_RESEAUX_ETUDIES):
    '''Cette fonction trie une liste en fonction de la présence d'une liste de réseau. Elle enregistre les items triés dans deux fichiers différents'''
    # Création des listes
    liste_inclue = []
    liste_exclue = []
    liste_exclue_api = []

    for gare in data:
        # Vérifie la présence d'au moins un network parmis une liste
        if gare["networks"] is None :
            liste_exclue_api.append(gare)
        elif any(reseau_id in [reseau["id"] for reseau in gare["networks"]] for reseau_id in LISTE_RESEAUX_ETUDIES):
            liste_inclue.append(gare)
        else:
            liste_exclue.append(gare)

    #Enregistrement des deux liste après trie
    enregistrer_json(liste_exclue_api,"data/_gare_frequentation_exclues_API.json")
    print(f"{len(liste_exclue_api)} gares ont été enregistrées dans data/gare_frequentation_exclues_API.json ")

    enregistrer_json(liste_inclue,"data/_gare_frequentation_inclues.json")
    print(f"{len(liste_inclue)} gares ont été enregistrées dans data/gare_frequentation_inclues.json ")

    enregistrer_json(liste_exclue,"data/_gare_frequentation_exclues.json")
    print(f"{len(liste_exclue)} gares ont été enregistrées dans data/gare_frequentation_exclues.json ")

In [42]:
def main():

    #Calcul du nombre de page
    nb_page = calcul_nombre_page()
    #Fais les requetes sur toutes les pages
    data_frequentation_brut = recuperer_toutes_pages(1)
    enregistrer_json(data_frequentation_brut,"test.json")
    liste_gares_reseaux = []
    # Ajout des réseaux à chaque gare
    gares_reseaux = ajout_reseau(data_frequentation_brut,liste_gares_reseaux,api_key)
    # Trie et enregistrement des gares
    trie_gare(gares_reseaux,LISTE_RESEAUX_ETUDIES)

In [43]:
main()

gare n°10 sur 50
gare n°20 sur 50
gare n°30 sur 50
gare n°40 sur 50
gare n°50 sur 50
1 gares ont été enregistrées dans data/gare_frequentation_exclues_API.json 
48 gares ont été enregistrées dans data/gare_frequentation_inclues.json 
1 gares ont été enregistrées dans data/gare_frequentation_exclues.json 
